In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [24]:
(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train','test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

In [25]:
ds_info

tfds.core.DatasetInfo(
    name='cifar10',
    full_name='cifar10/3.0.2',
    description="""
    The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.
    """,
    homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
    data_path='C:\\Users\\Nirajan\\tensorflow_datasets\\cifar10\\3.0.2',
    file_format=tfrecord,
    download_size=162.17 MiB,
    dataset_size=132.40 MiB,
    features=FeaturesDict({
        'id': Text(shape=(), dtype=string),
        'image': Image(shape=(32, 32, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=50000, num_shards=1>,
    },
    citation="""@TECHREPORT{Krizhevsky09learningmultiple,
        author = {Alex Krizhevsky},
      

In [26]:
def normalize_img(image, label):
    return tf.cast(image, tf.float32)/255.0, label

In [27]:
def augment(image, label):
    image = tf.image.resize(image, (32,32))

    if tf.random.uniform((), minval=0, maxval=1) <=0.1:
        image = tf.tile(tf.image.rgb_to_grayscale(image), [1,1,3])

    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.1, upper=0.2)
    image = tf.image.random_flip_left_right(image) # 50%
    # image = tf.image.random_flip_up_down(image) # 50%

    return image, label

In [28]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.map(augment, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(AUTOTUNE)

In [29]:
model = keras.Sequential(
    [
        layers.Input(shape=(32, 32, 3)),
        layers.Conv2D(8, 3, activation='relu'),
        layers.Conv2D(16, 3, activation='relu'),
        layers.MaxPool2D(),
        layers.Conv2D(32, 3, activation='relu'),
        layers.GlobalMaxPool2D(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10)
    ]
)

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 30, 30, 8)         224       
                                                                 
 conv2d_4 (Conv2D)           (None, 28, 28, 16)        1168      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 16)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 12, 12, 32)        4640      
                                                                 
 global_max_pooling2d_1 (Glo  (None, 32)               0         
 balMaxPooling2D)                                                
                                                                 
 dense_2 (Dense)             (None, 64)               

In [31]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy']
)

In [32]:
model.fit(
    ds_train,
    epochs = 5,
    validation_data = ds_test
)

Epoch 1/5
1563/1563 [==============================] - 16s 6ms/step - loss: 2.1196 - accuracy: 0.1961 - val_loss: 3.7912 - val_accuracy: 0.1234
Epoch 2/5
1563/1563 [==============================] - 9s 5ms/step - loss: 1.9819 - accuracy: 0.2495 - val_loss: 3.8374 - val_accuracy: 0.1742
Epoch 3/5
1563/1563 [==============================] - 9s 5ms/step - loss: 1.8980 - accuracy: 0.2909 - val_loss: 4.6807 - val_accuracy: 0.1853
Epoch 4/5
1563/1563 [==============================] - 9s 6ms/step - loss: 1.8436 - accuracy: 0.3145 - val_loss: 4.4297 - val_accuracy: 0.2083
Epoch 5/5
1563/1563 [==============================] - 9s 6ms/step - loss: 1.7891 - accuracy: 0.3389 - val_loss: 4.0063 - val_accuracy: 0.2361
